In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys

import fire

import csv
import sys
import os
import json
import matplotlib.pyplot as plt
import re
import string

import matplotlib.dates as mdates
import seaborn as sns

# to view all columns
pd.set_option("display.max.columns", None)

import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
import sys


import preprocessor as p

In [104]:
class Tweets():
    
    # class variables + tweepy authentication
    def __init__(self, intervention_date = '2020-04-01', earliest_date = '2020-01-01', post_date = '2020-07-01'):
        self.consumer_key = os.environ.get('consumer_key')
        self.consumer_secret = os.environ.get('consumer_secret')
        self.access_token = os.environ.get('access_token')
        self.access_token_secret = os.environ.get('access_token_secret')
        
        self.auth = OAuthHandler(self.consumer_key, self.consumer_secret)
        self.auth.set_access_token(self.access_token, self.access_token_secret)
        self.auth_api = API(self.auth, wait_on_rate_limit=True)
        
        self.intervention_date = intervention_date
        self.post_date = post_date
        self.earliest_date = earliest_date
        
#     # country id 
#     def get_placeid(self, country):
#         places = self.auth_api.geo_search(query=country, granularity="country")
#         place_id = places[0].id
#         return place_id
    
    # get tweets
    def get_tweets(self, hashtag, geocode):
        pre_intervention_tweets = tweepy.Cursor(self.auth_api.search , q=hashtag, geocode = geocode, since_id = self.earliest_date).items(300)
        post_intervention_tweets = tweepy.Cursor(self.auth_api.search , q=hashtag, geocode = geocode, since_id = self.post_date).items(300)
        during_intervention_tweets = tweepy.Cursor(self.auth_api.search , q=hashtag, geocode = geocode, since_id = self.intervention_date).items(300)
        
        return pre_intervention_tweets, during_intervention_tweets, post_intervention_tweets
    
    
    
#     def get_tweets(self, place_id):
#         # tweets before covid intervention
# #         pre_intervention_tweets = tweepy.Cursor(self.auth_api.search , q=self.keywords and ("place:%s" % place_id),  since_id = self.earliest_date).items(10000)
#         tweets = tweepy.Cursor(self.auth_api.search , q = self.keywords and ("place:%s" % place_id),  since_id = self.start_date).items(100)
#         pre_intervention_tweets = []
#         for tweet in tweets:
#             if tweet.created_at < self.end_date and tweet.created_at > self.start_date:
#                 pre_intervention_tweets.append(tweet)        
# #         tweets_ = tweepy.Cursor(self.auth_api.search , q = self.keywords and ("place:%s" % place_id),  since_id = self.start_date).items(100)
# #         during_intervention_tweets = []
# #         for tweet in tweets_:
# #             if tweet.created_at < endDate and tweet.created_at > startDate:
# #                 during_intervention_tweets.append(tweet)          
# #       tweets post covid intervention 
#         post_intervention_tweets = tweepy.Cursor(self.auth_api.search , q=self.keywords and ("place:%s" % place_id),  since_id = self.intervention_date).items(1000)
#         return pre_intervention_tweets, post_intervention_tweets
    
    # get tweet attributes ~ save to csv
    def get_var(self, country, tweets, tag, period = 'pre_covid'):

        alltweets = [] 
        alltweets.extend(tweets)

        outtweets = [[tweet.id_str, tweet.user.screen_name, tweet.created_at, tweet.text, tweet.favorite_count, tweet.retweet_count, tweet.entities, tweet.lang] for tweet in alltweets]

        #write to csv  
        with open(f'{country}_{period}_{tag}_tweets.csv', 'w') as f:
            writer = csv.writer(f)
            writer.writerow(["id", "user", "created_at","text", "favorites", "retweets", "entities", "language"])
            writer.writerows(outtweets)

        pass


In [105]:
# instantiate class
t = Tweets()

# geocodes
codes = {'KE' : "0.0236,37.9062,530km", 'NG' : "9.0820,8.6753, 750km", 
         'ZA' : "30.5595, 22.9375, 1000km", 'UG': "1.3733, 32.2903, 400km"}

kenya = "0.0236,37.9062,530km"
nigeria = "9.0820,8.6753, 750km"
south_africa = "30.5595, 22.9375, 1000km"
uganda = "1.3733, 32.2903, 400km"

codes

{'KE': '0.0236,37.9062,530km',
 'NG': '9.0820,8.6753, 750km',
 'ZA': '30.5595, 22.9375, 1000km',
 'UG': '1.3733, 32.2903, 400km'}

In [106]:
tags = ['AirPollution', 'Environment', 'Ozone Layer', 'Global Warming', 'Climate Change', 'Greenhouse Gases', 'Trees', 'Carbon',
        'Aerosals', 'Air', 'Save the planet', 'Factories', 'Hygroscopicity', 'Inversion', 'Sulfur', 'AIRS', 'ecosystem', 'Hydrochlorofluorocarbon',
        'hydrocarbon', 'TAC', 'zero', 'pollutant', '#air', '#pollution', '#airpollution', '#coal', '#particles', '#smog', '#cleanair',
       '#airqualityindex', '#climatechange', '#airquality', '#globalwarming', '#airpollutionawareness', '#airpollutioncontrol',
       '#CleanEnergy', '#saveearth']
len(tags)

37

change country code to your country's

In [107]:
# get tweets
pre_bowl = []
during_bowl = []
post_bowl = []

for hashtag in tags:
    KE_pre_intervention_tweets, KE_during_intervention_tweets, KE_post_intervention_tweets = t.get_tweets(hashtag, codes['KE'])
    pre_bowl.append(KE_pre_intervention_tweets)
    during_bowl.append(KE_during_intervention_tweets)
    post_bowl.append(KE_post_intervention_tweets)
    
# NG_pre_intervention_tweets, NG_post_intervention_tweets = get_tweets(ids['NG'])
# UG_pre_intervention_tweets, UG_post_intervention_tweets = get_tweets(ids['UG'])
# ZA_pre_intervention_tweets, ZA_post_intervention_tweets = get_tweets(ids['ZA'])

In [113]:
# csv files of kenya pre tweets
names = [*range(len(tags))]
names =  [str(x) for x in names]

for tweets, n in zip(pre_bowl, names):
    t.get_var('KE', tweets, n, period = 'pre_int')
    
# csv files of kenya during tweets
names = [*range((len(tags)))]
names =  [str(x) for x in names]

for tweets, n in zip(during_bowl, names):
    t.get_var('KE', tweets, n, period = 'during_int')
    
    
# csv files of kenya post tweets
names = [*range(len(tags))]
names =  [str(x) for x in names]

for tweets, n in zip(post_bowl, names):
    t.get_var('KE', tweets, n, period = 'post_int')